# XGBoost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt

## Carga de datos

In [ ]:
inPath = "../../1 Preprocesado"

In [ ]:
# Ficheros de datos
polen = pd.read_csv(f"{inPath}/polen/gramineasGETA.csv")
tiempo = pd.read_csv(f"{inPath}/tiempo/Getafe.csv")

# Unimos los datos
datos = pd.merge(polen, tiempo, on='fecha')

In [ ]:
# Constantes
ANTICIPACION = 1 # Número de días de anticipación con los que se predice
NUM_DIAS = 15   # Número de días en base a los que se predice
MODEL_FILE_NAME = f'XGBoost{NUM_DIAS}_{ANTICIPACION}.pkl' #Archivo de guardado del modelo

Adaptación de los datos

In [ ]:
### Prepación de los datos para el modelo de Random Forest
# Ordenamos los datos por fecha
datos = datos.sort_values('fecha')

# Creamos las columnas con los datos de los días anteriores
for i in range(1, NUM_DIAS + 1):
    for columna in ['granos_de_polen', 'prec', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol']:
        # Supress SettingWithCopyWarning
        datos = datos.copy()
        datos[f'{columna}_{i}'] = datos[columna].shift(i)

# Eliminamos los días que no tienen todos los datos
datos = datos.dropna()

# Posibles mejoras

In [ ]:
# Quedarse solo con los meses de mayo a junio (todos los años)
# (periodo de polinización de las gramíneas)
# datos = datos[(datos['fecha'].str[5:7] == '05') | (datos['fecha'].str[5:7] == '06')]

In [ ]:
# Añadir columna de numero de semana (calculo a mano)
datos['semana'] = pd.to_datetime(datos['fecha']).dt.dayofyear // 7

In [ ]:
# # ['prec', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol']
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('prec_')])
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('tmin_')])
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('tmax_')])
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('dir_')])
# # datos = datos.drop(columns=[col for col in datos.columns if col.startswith('velmedia_')])
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('racha_')])
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('sol_')])

# datos = datos.drop(columns=['tmin', 'dir', 'racha', 'sol'])
# datos = datos.drop(columns=['prec', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol'])

In [ ]:
# Quitar columnas que empiezan por granos_de_polen_
# datos = datos.drop(columns=[col for col in datos.columns if col.startswith('granos_de_polen_')])

In [ ]:
# Eliminamos la columna de fecha
datos = datos.drop(columns=['fecha'])

# Datos en entrenamiento y test

In [ ]:
# Separamos los datos en entrenamiento y test
X = datos.drop(columns=['granos_de_polen'])
y = datos['granos_de_polen']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Normalizamos los datos (no es necesario para Random Forest)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# # Escalar los datos 
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

In [ ]:
# Visualización de los datos (sepamos los datos de entrenamiento y test)
import matplotlib.pyplot as plt
plt.plot(y_train.index, y_train, label='Train')
plt.plot(y_test.index, y_test, label='Test')
plt.legend()
plt.show()

## Entrenamiento (o carga) del modelo

In [ ]:
model = None

In [ ]:
# Entrenar el modelo
model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.25, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)

In [ ]:
# Evaluación del modelo utilizando el conjunto de prueba
y_pred = model.predict(X_test)

## Métricas

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

rmse = np.sqrt(mse)
print('RMSE:', rmse)

mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

r2 = r2_score(y_test, y_pred)
print('R2:', r2)

#### Métricas con variables ordinales (alto-medio-bajo)

In [ ]:
# Define the classification thresholds
low_threshold = 25
medium_threshold = 50

def classify_label(value):
    if value < low_threshold:
        return 'Low'
    elif value < medium_threshold:
        return 'Medium'
    else:
        return 'High'

# Classify the predictions based on the thresholds
y_true = [classify_label(value) for value in y_test]  # Replace classify_label with your own function
y_pred_clas = [classify_label(value) for value in y_pred]  # Replace classify_label with your own function

Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix

# Create the confusion matrix
labels = ['Low', 'Medium', 'High']
cm = confusion_matrix(y_true, y_pred_clas, labels=labels)

# Display the confusion matrix with a heatmap
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

Métricas (categóricas)

In [ ]:
# Mostrar métricas de clasificación
from sklearn.metrics import classification_report

# Display the classification report
print(classification_report(y_true, y_pred_clas))

## Visualización de los resultados

In [ ]:
# Importancia de las características
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, X.columns[indices[f]], importances[indices[f]]))

In [ ]:
# Importancia de las características más importantes (las 10 primeras)
import seaborn as sns
color_palette = sns.color_palette("Spectral", len([X.columns[i] for i in indices[:10]]))
plt.figure(figsize=(15, 7))
sns.barplot(x=[X.columns[i] for i in indices[:10]], y=importances[indices[:10]], palette=color_palette, hue=[X.columns[i] for i in indices[:10]])
plt.title('Importancia de las características')
plt.show()

In [ ]:
# Gráfico de dispersión de los valores reales frente a los predichos
plt.scatter(y_test, y_pred)
plt.xlabel('Valores reales')
plt.ylabel('Valores predichos')
plt.title('Valores reales vs Valores predichos')
# Línea de tendencia
z = np.polyfit(y_test, y_pred, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), color='darkgreen')
# Línea de 45 grados
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='black')
plt.show()

In [ ]:
# Gráfico de la predicción con puntos
plt.plot(y_test.index, y_test, label='Real', color='blue', marker='o')
plt.plot(y_test.index, y_pred, label='Predicción', color='red', marker='x')
# Recortar a fragmentos cortos para poder ver lo que pasa
# plt.xlim(3175, 3220)
# plt.xlim(3520, 3560)
plt.xlim(4080, 4130)
# Dibujar líneas horizontales para los umbrales
plt.axhline(y=low_threshold, color='green', linestyle='--', label='Low threshold')
plt.axhline(y=medium_threshold, color='orange', linestyle='--', label='Medium threshold')
plt.legend()
plt.show()

## Guardado del modelo

In [ ]:
# Guardar el modelo
joblib.dump(model, MODEL_FILE_NAME)